<a href="https://colab.research.google.com/github/attackgnome/pytorch_project/blob/master/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>